# Two Rabin

> CRYPTO | 360 | 20 Solves
> 
> Two problems, two ciphertexts each, and Rabin.
> 
> Attachment: two_rabin.tar.gz (md5 3bc8aa7b65eddb4cffbf99df49c8a606)

This is a sagemath notebook

## Easy

- We want to find the flag $m$, and we have this information:
  - Number of bytes of $m$
  - $n$, a product of two large primes
  - $B$, another large prime
  - $c_1 = m \cdot (m + B) \mod n$
  - $c_2 = \text{pad}(m) \cdot (\text{pad}(m) + B) \mod n$

In [1]:
bs = 128
flag1_len = 98
n = 105663510238670420757255989578978162666434740162415948750279893317701612062865075870926559751210244886747509597507458509604874043682717453885668881354391379276091832437791327382673554621542363370695590872213882821916016679451005257003326444660295787578301365987666679013861017982035560204259777436442969488099
B = 12408624070212894491872051808326026233625878902991556747856160971787460076467522269639429595067604541456868927539680514190186916845592948405088662144279471
c1 = 47149257341850631803344907793040624016460864394802627848277699824692112650262968210121452299581667376809654259561510658416826163949830223407035750286554940980726936799838074413937433800942520987785496915219844827204556044437125649495753599550708106983195864758161432571740109614959841908745488347057154186396
c2 = 38096143360064857625836039270668052307251843760085437365614169441559213241186400206703536344838144000472263634954875924378598171294646491844012132284477949793329427432803416979432652621257006572714223359085436237334735438682570204741205174909769464683299442221434350777366303691294099640097749346031264625862

- Since we know number of bytes in $m$, we can predict the effect of $\text{pad}()$
$$
\text{pad}(m) = p_1 m + p_0
$$

In [2]:
from Crypto.Util.Padding import pad
from Crypto.Util.number import bytes_to_long

fakeflag = b"?" * flag1_len
fakeflag_pad = pad(fakeflag, bs)

print(f"""
fakeflag     : {fakeflag}
fakeflag_pad : {fakeflag_pad}
""".strip())

fakeflag_int = bytes_to_long(fakeflag)
fakeflag_pad_int = bytes_to_long(fakeflag_pad)

pad_len = bs - len(fakeflag)

p_1 = bytes_to_long(b"\x01" + b"\x00" * pad_len)
p_0 = bytes_to_long(bytes([pad_len]) * pad_len)

assert(fakeflag_pad_int == p_1 * fakeflag_int + p_0)

fakeflag     : b'??????????????????????????????????????????????????????????????????????????????????????????????????'
fakeflag_pad : b'??????????????????????????????????????????????????????????????????????????????????????????????????\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e'


- We can now express $c_2$ in terms of m
$$
\begin{align*}
c_2 & = \text{pad}(m) \cdot (\text{pad}(m) + B) \mod n \\
& = (p_1 m + p_0) \cdot (p_1 m + p_0 + B) \mod n & \text{(Expand pad)} \\
\end{align*}
$$
- With this we can solve for m

In [3]:
P.<x> = PolynomialRing(Zmod(n))
eq1 = x * (x + B) - c1
eq2 = (p_1 * x + p_0) * (p_1 * x + p_0 + B) - c2
print("eq1 :")
print(eq1)
print("eq2 :")
print(eq2)

eq1 :
x^2 + 12408624070212894491872051808326026233625878902991556747856160971787460076467522269639429595067604541456868927539680514190186916845592948405088662144279471*x + 58514252896819788953911081785937538649973875767613320902002193493009499412602107660805107451628577509937855337945947851188047879732887230478633131067836438295364895637953252968736120820599842382910093956994037994711460635013879607507572845109587680595105501229505246442120908367075718295514289089385815301703
eq2 :
3121748550315992231381597229793166305748598142664971150859156959625371738819765620120306103063491971159826931121406622895447975679288285306290176*x^2 + 21924141016394061015849738775213955608922446807172134393786137357364766362969263060582515058406042208001615778549864215806651329531781078738537563580505965426840774552416571706884458095099675848747569895256516999794193959223296*x + 675673668786055631314199503083101103591828964023305113846657238761423988216786756668023341141831668881399211125777538333347990261

In [4]:
eq3 = eq2 % eq1
print("eq3 :")
print(eq3)

eq3 :
105663510199933816554652759199351628402075784331510241625398555689429769864745822693191441581288206097894236953776461039479552598793880953246913484564397912215824473068773375504671736599268883821808568121155440968884406414495765962277188937932926172140803259836795064264374492398241742154001737157505362934499*x + 75188846410738192156711504119945160827657652472978311413764975866246887979398722951465315481694742319030632356591855817520816868812499690071779102981808295940224058962113431191383203006290708150218866689196140942977016824385510083865500445449055915257633497282129559342075029579935804699054407501848478588736


In [5]:
easy_ans = -eq3[0] / eq3[1]
assert(eq1.subs(x=easy_ans) == 0)
print(bytes.fromhex(hex(easy_ans)[2:]))

b'ACSC{Rabin_cryptosystem_was_published_in_January_1979_ed82c25b173f38624f7ba16247c31d04ca22d8652da4'


## Hard

- We want to find the flag $m$, and we have this information:
  - Number of bytes of $m$
  - $n$, a product of two large primes
  - $B$, another large prime
  - $c_1 = (p_1 m + r_1) \cdot (p_1 m + r_1 + B) \mod n$
  - $c_2 = (p_1 m + r_2) \cdot (p_1 m + r_2 + B) \mod n$
  - Where $p_1$ is same as before due to padding; and
  - Where $r_1$, $r_2$ are random numbers less than $p_1$

In [6]:
flag2_len = 98
hard_c1 = 73091191827823774495468908722773206641492423784400072752465168109870542883199959598717050676487545742986091081315652284268136739187215026022065778742525832001516743913783423994796457270286069750481789982702001563824813913547627820131760747156379815528428547155422785084878636818919308472977926622234822351389
hard_c2 = 21303605284622657693928572452692917426184397648451262767916068031147685805357948196368866787751567262515163804299565902544134567172298465831142768549321228087238170761793574794991881327590118848547031077305045920819173332543516073028600540903504720606513570298252979409711977771956104783864344110894347670094

- Notably, the difference between $p_1 m + r_1$ and $p_1 m + r_2$ is small compared to the modulus $n$, with similar magnitude to $p_1$
- Rearrange with new values $x$ and $y$ such that
$$
\begin{align*}
x & = p_1 m + r_1 \mod n \\
y & = (p_1 m + r_2) - (p_1 m + r_1) \mod n \\
& = r_2 - r_1 \mod n
\end{align*}
$$
- $y$ is small, we can try to use coppersmith for small roots
- Plugging in $x$ and $y$ into equations for $c_1$ and $c_2$
$$
\begin{align*}
c_1 & = x \cdot (x + B) \mod n \\
c_2 & = (x + y) \cdot (x + y + B) \mod n
\end{align*}
$$

In [7]:
P.<x, y> = PolynomialRing(Zmod(n))
eq1 = x * (x + B) - hard_c1 # equals 0
eq2 = (x + y) * (x + y + B) - hard_c2 # equals 0
print("eq1 :")
print(eq1)
print("eq2 :")
print(eq2)

eq1 :
x^2 + 12408624070212894491872051808326026233625878902991556747856160971787460076467522269639429595067604541456868927539680514190186916845592948405088662144279471*x + 32572318410846646261787080856204956024942316378015875997814725207831069179665116272209509074722699143761418516191806225336737304495502427863603102611865547274575088524007903387877097351256293620213800889511881258091202765903377436871565697503915972049872818832243893928982381163116251731281850814208147136710
eq2 :
x^2 + 2*x*y + y^2 + 12408624070212894491872051808326026233625878902991556747856160971787460076467522269639429595067604541456868927539680514190186916845592948405088662144279471*x + 12408624070212894491872051808326026233625878902991556747856160971787460076467522269639429595067604541456868927539680514190186916845592948405088662144279471*y + 84359904954047763063327417126285245240250342513964685982363825286553926257507127674557692963458677624232345793207892607060739476510418988054526112805070151188853661675997752

In [8]:
eq3 = eq2 - eq1 # equals 0
print("eq3 :")
print(eq3)

eq3 :
2*x*y + y^2 + 12408624070212894491872051808326026233625878902991556747856160971787460076467522269639429595067604541456868927539680514190186916845592948405088662144279471*y + 51787586543201116801540336270080289215308026135948809984549100078722857077842011402348183888735978480470927277016086381724002172014916560190923010193204603914278573151989849199804575942695950901934758905396955643005640581004111747103160206252875094921914976857169805675166659046963203689113582511340474681295


- From `eq3` we have an expression for $xy$ in terms of $y$
$$
\begin{align*}
0 & = 2 x y + y^2 + \ldots \mod n \\
x y & = -\frac{1}{2} (y^2 + \ldots) \mod n & \text{(Rearrange } xy \text{)} \\
\end{align*}
$$

In [9]:
xy = (2 * x * y - eq3) / 2 # equals xy
print("xy :")
print(xy)

xy :
52831755119335210378627994789489081333217370081207974375139946658850806031432537935463279875605122443373754798753729254802437021841358726942834440677195689638045916218895663691336777310771181685347795436106941410958008339725502628501663222330147893789150682993833339506930508991017780102129888718221484744049*y^2 + 52831755119335210378627994789489081333217370081207974375139946658850806031432537935463279875605122443373754798753729254802437021841358726942834440677195683433733881112448417755310873147758064872408343940328567482877522445995464394740528402615350359986879954559369569666673413897559357305655686173890412604314*y + 26937961847734651977857826654448936725563357013233569382865396619489377492511532234289187931237133203138291160245686063940435935833900446847372935580593387680906629642900739091434489339423206234380415983408463589455188049223446754950083119203710346328193194565248436669347179467536178257573097462551247403402


- We can further manipulate `eq1`
$$
\begin{align*}
0 & = x \cdot (x + B) - c_1\mod n \\
& = x^2 + B x - c_1 \mod n \\
& = x^2 y^2 + B x y^2 - c_1 y^2 \mod n & \text{(Multiply by } y^2 \text{)} \\
& = (-\frac{1}{2} (y^2 + \ldots))^2 + B y \cdot (-\frac{1}{2} (y^2 + \ldots)) - c_1 y^2 \mod n & \text{(Substitute } xy \text{)} \\
\end{align*}
$$

In [10]:
eq4 = xy ^ 2 + B * xy * y - hard_c1 * y ^ 2 # equals 0
print("eq4 :")
print(eq4)

eq4 :
26415877559667605189313997394744540666608685040603987187569973329425403015716268967731639937802561221686877399376864627401218510920679363471417220338597844819022958109447831845668388655385590842673897718053470705479004169862751314250831611165073946894575341496916669753465254495508890051064944359110742372025*y^4 + 46388501413148104768752934655281598750824534273679767757538783362168949475914075311729363798869456947438595342868190388884610971919396387844220351199123138206842857221926512147996174710657711618586853097289245939411711972094455973280277486344454703380273192998069154060020096378543084781099650495946688839422*y^2 + 92020846942869483880287239041132433098227272744587627695915579378991133800180762512466831285103747875899289678719823184241174232164008770284671164843841090107431103181572986730809954645499280109047957101035694534676659306168944268030272683840665016850028031523373964876152256501786671085723161402040868954766


- We have a polynomial only in $y$
- Since $y$ is small, we can use `.small_roots()`

In [11]:
deg = eq4.degree(y)
coeffs = [eq4[0, i] for i in range(deg + 1)]

P.<x> = PolynomialRing(Zmod(n), 1)
y_poly = sum([coeff * x ^ i for i, coeff in enumerate(coeffs)])
print("y_poly :")
print(y_poly)

y_poly :
26415877559667605189313997394744540666608685040603987187569973329425403015716268967731639937802561221686877399376864627401218510920679363471417220338597844819022958109447831845668388655385590842673897718053470705479004169862751314250831611165073946894575341496916669753465254495508890051064944359110742372025*x^4 + 46388501413148104768752934655281598750824534273679767757538783362168949475914075311729363798869456947438595342868190388884610971919396387844220351199123138206842857221926512147996174710657711618586853097289245939411711972094455973280277486344454703380273192998069154060020096378543084781099650495946688839422*x^2 + 92020846942869483880287239041132433098227272744587627695915579378991133800180762512466831285103747875899289678719823184241174232164008770284671164843841090107431103181572986730809954645499280109047957101035694534676659306168944268030272683840665016850028031523373964876152256501786671085723161402040868954766


In [12]:
load("coppermsith.sage")

y_actual = small_roots(y_poly, (p_1,), m=8)[0][0]

print("y_actual :")
print(y_actual)

# For the answer, r_2 for the full solution is actually smaller than r_1, so we flip y_actual
y_actual = -y_actual

y_actual :
1637558660573652475698054766420163959191730746581158985657024969935597275


- From here on we repeat small solution

In [13]:
P.<x> = PolynomialRing(Zmod(n))
eq1 = x * (x + B) - hard_c1
eq2 = (x + y_actual) * (x + y_actual + B) - hard_c2
eq3 = eq2 % eq1

hard_ans = -eq3[0] / eq3[1]
assert(eq1.subs(x=hard_ans) == 0)
print(bytes.fromhex(hex(hard_ans)[2:]))

b'a1d701b0966ffa10a4d1_ec0c177f446964ca9595c187869312b2c0929671ca9b7f0a27e01621c90a9ac255_wow_GJ!!!}\xfa\xc4\xdb\xa6\xa1\xcd\xc6\xf9:\x08\xbb\xf1\xa9\x1e\xa2\xec4\x88@Q\x15\x85U\x181A\xd7V\x81\x1f'


In [14]:
print(bytes.fromhex(hex(easy_ans)[2:]) + bytes.fromhex(hex(hard_ans)[2:]))

b'ACSC{Rabin_cryptosystem_was_published_in_January_1979_ed82c25b173f38624f7ba16247c31d04ca22d8652da4a1d701b0966ffa10a4d1_ec0c177f446964ca9595c187869312b2c0929671ca9b7f0a27e01621c90a9ac255_wow_GJ!!!}\xfa\xc4\xdb\xa6\xa1\xcd\xc6\xf9:\x08\xbb\xf1\xa9\x1e\xa2\xec4\x88@Q\x15\x85U\x181A\xd7V\x81\x1f'


Flag: `ACSC{Rabin_cryptosystem_was_published_in_January_1979_ed82c25b173f38624f7ba16247c31d04ca22d8652da4a1d701b0966ffa10a4d1_ec0c177f446964ca9595c187869312b2c0929671ca9b7f0a27e01621c90a9ac255_wow_GJ!!!}`